In [1]:
#Dependancies 
import numpy as np 
import pandas as pd
import seaborn as sns 
import cv2
import skimage.io as io
import keras
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls 
py.init_notebook_mode(connected=True)
import random 
import shutil

from skimage.io import imread, imshow
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

from subprocess import check_output
print(check_output(["ls", "/home/kray/Practicum/train"]).decode("utf8"))

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from keras import backend as K
from skimage.transform import resize

Using TensorFlow backend.


Type_1
Type_2
Type_3



In [2]:
#Split data into training and validation sets, keeping file organization. 
allFiles = glob.glob('/home/kray/Practicum/train/*/*')

split_point = int(round(0.7*len(allFiles)))
random.shuffle(allFiles)

train_list = allFiles[:split_point]
valid_list = allFiles[split_point:]
print('Train images: {}'.format(len(train_list)))
print('Validation images {}'. format(len(valid_list)))

# Divy them into their own directories.
train_data_path = '/home/kray/Practicum/train_data'
validation_data_path = '/home/kray/Practicum/valid_data'
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
if not os.path.exists(validation_data_path):
    os.makedirs(validation_data_path)

for fpath in train_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([train_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))
    shutil.copyfile(fpath, '/'.join([train_data_path] +basename))
for fpath in valid_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([validation_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))    
    shutil.copyfile(fpath, '/'.join([validation_data_path] +basename))

Train images: 1036
Validation images 444


In [69]:
K.set_image_dim_ordering('tf')

# Create numpy array
train_image_list = list()
train_label_list = list()
for image in train_list[:10]:
    im = cv2.imread(image)
    im = resize(im, (100, 100), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    train_label_list.append(im_type.split('Type_')[1])
    train_image_list.append(im)
print(train_label_list)
print(train_image_list)

# Reshape
#train_list = train_list.reshape(train_list[0], 100, 100, 3).astype('float32')
#valid_list = Reshape(valid_list[0], 100, 100, 3).astype('float32')

['3', '1', '2', '2', '2', '3', '3', '2', '3', '1']
[array([[[ 0.10172549,  0.08337255,  0.11866666],
        [ 0.08348392,  0.07956235,  0.09524862],
        [ 0.08373334,  0.06412549,  0.0714196 ],
        ..., 
        [ 0.42094117,  0.32749805,  0.30396864],
        [ 0.44738823,  0.37287843,  0.31555766],
        [ 0.41938353,  0.31577569,  0.27036393]],

       [[ 0.1031498 ,  0.10591529,  0.13898039],
        [ 0.09670588,  0.08494118,  0.12023529],
        [ 0.09523921,  0.06408627,  0.09689412],
        ..., 
        [ 0.44633725,  0.33770981,  0.33339608],
        [ 0.42282352,  0.34760785,  0.32049099],
        [ 0.42232785,  0.34089568,  0.31578511]],

       [[ 0.12930197,  0.1253804 ,  0.16459608],
        [ 0.12569411,  0.10501176,  0.12523137],
        [ 0.16721569,  0.12407843,  0.13298039],
        ..., 
        [ 0.42941177,  0.34117648,  0.35137254],
        [ 0.40574902,  0.31149021,  0.31149021],
        [ 0.41969413,  0.32557648,  0.32557648]],

       ..., 
     

In [57]:
test_image_list = list()
test_label_list = list()
for image in valid_list[:10]:
    im = cv2.imread(image)
    im = resize(im, (100, 100), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    test_label_list.append(im_type.split('Type_')[1])
    test_image_list.append(im)
print(test_label_list)
print(test_image_list)

['2', '3', '2', '2', '2', '3', '2', '1', '3', '2']
[array([[[ 0.08126588,  0.06227451,  0.06496314],
        [ 0.0808502 ,  0.04947765,  0.04947765],
        [ 0.09963137,  0.06534902,  0.04932549],
        ..., 
        [ 0.00789804,  0.01064314,  0.00352941],
        [ 0.00473882,  0.0142949 ,  0.00688941],
        [ 0.01975373,  0.01175373,  0.00783216]],

       [[ 0.15060079,  0.10354196,  0.08001255],
        [ 0.17835294,  0.14278588,  0.10762823],
        [ 0.21312942,  0.17698823,  0.12333333],
        ..., 
        [ 0.0356    ,  0.01991373,  0.01599216],
        [ 0.02554823,  0.00986196,  0.00594039],
        [ 0.00347608,  0.01960784,  0.01176471]],

       [[ 0.23258039,  0.19744314,  0.14241569],
        [ 0.27781177,  0.22055687,  0.1511451 ],
        [ 0.30615687,  0.25478432,  0.16968627],
        ..., 
        [ 0.07239216,  0.05815686,  0.03854902],
        [ 0.05094902,  0.03254902,  0.00446275],
        [ 0.0317647 ,  0.02345882,  0.0027451 ]],

       ..., 
     

In [4]:
print(im.shape)

(100, 100, 3)


In [70]:
X_train = np.array(train_image_list)
Y_train = np.array(train_label_list)

#dimensions of images
img_width, img_height = 100, 100

#one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
num_classes = Y.shape[1]

In [71]:
X_test = np.array(test_image_list)
Y_test = np.array(test_label_list)

#one hot encode outputs
Y_test = np_utils.to_categorical(Y_test)


In [62]:
# Simple CNN model.
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

def simple_cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(Y.shape[1], activation = 'softmax'))
    
# Compile model
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy']) 
    return model

# Build model
model = simple_cnn_model()

In [72]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(10, 100, 100, 3)
(10, 4)
(10, 100, 100, 3)
(10, 4)


In [73]:
# Fit the model
model.fit(X_train, Y_train, batch_size = 32, nb_epoch = 5, verbose = 1)
#for idx,im in enumerate(train_image_list):
#    model.fit(im, train_label_list[idx], batch_size=32, nb_epoch=5, verbose=1)

# Final evaluation of the model
#scores = model.evaluate(train_data_path, validation_data_path, verbose = 0)
scores = model.evaluate(X_test, Y_test, verbose = 0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/5
10/10 [==============================] - 0s - loss: 1.2872 - acc: 0.4000
Epoch 2/5
10/10 [==============================] - 0s - loss: 3.7129 - acc: 0.4000
Epoch 3/5
10/10 [==============================] - 0s - loss: 3.9518 - acc: 0.4000
Epoch 4/5
10/10 [==============================] - 0s - loss: 4.4568 - acc: 0.2000
Epoch 5/5
10/10 [==============================] - 0s - loss: 1.8806 - acc: 0.4000
CNN Error: 40.00%
